In [37]:
import bs4
import tqdm
import glob

In [38]:
def read_file(file):
    with open(file,"r") as f:
        soup = bs4.BeautifulSoup(f.read(),features="lxml")
    return soup

In [39]:
def is_index_or_header(file):
    return file in ["docs/index.html","docs/header.html"]

In [40]:
def format_last_updated(soup):
    p = soup.select(".dateGenerated")[0]
    p.string = f"Last updated on {p.text.strip().split('on')[1]}"

In [50]:
def make_search_button_clickable(soup):
    return

In [41]:
def is_book(file):
    # checking if path contains docs/book_0
    return file.find("docs/book_0/") != -1

In [42]:
def fix_book_title(soup):
    author_t = " div.fullEntry_title > h1 > small > em > small > em > a"
    title_t = " div.fullEntry_title > h1"
    p = soup.select(".x_container")[0]
    author = p.select(author_t)[0].text
    title = p.select(title_t)[0].text.split("\n")[1].strip()
    soup.head.title.string = f"{title}, By {author} | {soup.head.title.string}"

In [52]:
def is_author(file):
    # checking if path contains docs/author_0/
    return file.find("docs/author_0/") != -1  

In [53]:
def fix_author_title(soup):
    pass

In [43]:
def is_html(file):
    # checking last four chars of extention
    return file[-4:]=="html" 

In [44]:
def add_analytics(soup):
    analytics_code = '''<!-- Global site tag (gtag.js) - Google Analytics -->
                    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-101184475-7"></script>
                    <script>
                      window.dataLayer = window.dataLayer || [];
                      function gtag(){dataLayer.push(arguments);}
                      gtag('js', new Date());

                      gtag('config', 'UA-101184475-7');
                    </script>'''
    #print("add_analytics incomplete")
    return

In [45]:
def remove_footer(soup):
    soup.find(id="footer").decompose()

In [46]:
def write_file(file,soup):
    with open(file,"w") as f:
        f.write(str(soup))   

In [47]:
files = list(glob.glob("docs/*/*.html"))
files.extend(glob.glob("docs/*.html"))

In [48]:
for f in tqdm.tqdm(files):
    errs = []
    s = read_file(f)
    
    if is_index_or_header(f):
        try:
            format_last_updated(s)
            if f == "docs/index.html":
                make_search_button_clickable(s)
        except Exception as e:
            errs.append(("failed format_last_updated",e))
    
    if is_book(f):
        try:
            fix_book_title(s)
        except Exception as e:
            errs.append(("failed fix_book_title",e))
    
    if is_author(f):
        try:
            fix_author_title(s)
        except Exception as e:
            errs.append(("failed fix_author_title",e))
        
    
    if is_html(f):
        try:
            add_analytics(s)
        except Exception as e:
            errs.append(("failed add_analytics",e))
            
        if f not in  ["docs/search/search.html","docs/header.html"]:
            try:
                remove_footer(s)
            except Exception as e:
                errs.append(("failed remove_footer",e))
       
    
    if len(errs) == 0:
        write_file(f,s)
    else:
        print(f"Failed {f}",errs)
        

100%|██████████| 881/881 [00:07<00:00, 120.03it/s]
